## Import

In [1]:
import pandas as pd
import numpy as np ; np.random.seed(2024)
import warnings;warnings.filterwarnings(action='ignore')

# Feature Engineering
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

# Modeling
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor, BayesianRidge
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

## Read data

In [2]:
train = pd.read_csv('data/stayed_train.csv')
test = pd.read_csv('data/stayed_test.csv')
print('학습데이터 수:', train.shape)
print('평가데이터 수:', test.shape)

학습데이터 수: (127739, 20)
평가데이터 수: (40148, 19)


In [3]:
'''
데이터 설명
- sessionID : 세션 ID
- userID : 사용자 ID
- TARGET : 세션에서 발생한 총 조회수
- browser : 사용된 브라우저
- OS : 사용된 기기의 운영체제
- device : 사용된 기기
- new : 첫 방문 여부 (0: 첫 방문 아님, 1: 첫 방문)
- quality : 세션의 질 (거래 성사를 기준으로 측정된 값, 범위: 1~100)
- duration : 총 세션 시간 (단위: 초)
- bounced : 이탈 여부 (0: 이탈하지 않음, 1: 이탈함)
- transaction : 세션 내에서 발생의 거래의 수
- transaction_revenue : 총 거래 수익
- continent : 세션이 발생한 대륙
- subcontinent : 세션이 발생한 하위 대륙
- country : 세션이 발생한 국가
- traffic_source : 트래픽이 발생한 소스
- traffic_medium : 트래픽 소스의 매체
- keyword : 트래픽 소스의 키워드, 일반적으로 traffic_medium이 organic, cpc인 경우에 설정
- referral_path : traffic_medium이 referral인 경우 설정되는 경로
'''
display(train.head())

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword1,keyword2,referral_path1,referral_path2
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,(not set),(not set),(not set)
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,(not set),(not set),(not set)
2,SESSION_000007,USER_000007,5.0,Chrome,Macintosh,desktop,1,1.0,64.0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,(not set),(not set),(not set)
3,SESSION_000008,USER_000008,5.0,Firefox,Linux,desktop,1,1.0,60.0,0.0,0.0,Americas,South America,Brazil,youtube.com,referral,(not set),(not set),Category5,0002
4,SESSION_000009,USER_000009,3.0,Chrome,Macintosh,desktop,1,2.0,579.0,0.0,0.0,Americas,Northern America,United States,google,ad,Category1,000,(not set),(not set)


## <font color='forestgreen'> Data Cleansing

In [4]:
# browser 내 device 정보가 섞여 범주값이 다양하다. 학습데이터를 기반으로 재구성한다.
# 결측치로 생각하는 (not set)도 포함한다.
unclear_browser = ["Mozilla Compatible Agent", "MRCHROME", "+Simple Browser",
                   'SeaMonkey', 'osee2unifiedRelease', 'YE', 'Browser', 'starmaker', '(not set)']

train['browser'] = train['browser'].apply(lambda x: 'Safari' if 'Safari' in x else
                                                     ('Opera' if 'Opera' in x else
                                                     ('Amazon' if 'Amazon' in x else
                                                     ('BROWSER' if x in unclear_browser else x))))
test['browser'] = test['browser'].apply(lambda x: 'Safari' if 'Safari' in x else
                                                     ('Opera' if 'Opera' in x else
                                                     ('Amazon' if 'Amazon' in x else
                                                     ('BROWSER' if x in unclear_browser else x))))

In [5]:
# 같은 이유로 OS도 재구성한다.
train['OS'] = train['OS'].apply(lambda x: 'Windows' if 'Windows' in x else
                                           ('Nintendo' if 'Nintendo' in x else                                
                                           ('OS' if x in ['OS/2','(not set)'] else x)))
test['OS'] = test['OS'].apply(lambda x: 'Windows' if 'Windows' in x else
                                           ('Nintendo' if 'Nintendo' in x else                                
                                           ('OS' if x in ['OS/2','(not set)'] else x)))

In [6]:
# traffic_source를 지역 도메인을 제외해 재구성한다.
# google엔 analytics, mail, groups, adwords & ads, optimize, docs, earth 등이 섞여있다.
train['traffic_source'] = train.traffic_source.str.replace('.com|.net|.org', '')\
                          .apply(lambda x: 'yahoo' if '.yahoo' in x else
                                            ('facebook' if '.facebook' in x else
                                            ('youtube' if '.youtube' in x else
                                            ('pinterest' if '.pinterest' in x else
                                            ('edu' if '.edu' in x else
                                            ('ask' if '.ask' in x else
                                            ('reddit' if '.reddit' in x else
                                            ('google' if 'google' in x else x))))))))

test['traffic_source'] = test.traffic_source.str.replace('.com|.net|.org', '')\
                         .apply(lambda x: 'yahoo' if '.yahoo' in x else
                                            ('facebook' if '.facebook' in x else
                                            ('youtube' if '.youtube' in x else
                                            ('pinterest' if '.pinterest' in x else
                                            ('edu' if '.edu' in x else
                                            ('ask' if '.ask' in x else
                                            ('reddit' if '.reddit' in x else
                                            ('google' if 'google' in x else x))))))))

## <font color='forestgreen'> Feature Engineering

### Feature Generation

In [7]:
# stayed 데이터는 변수를 생성한다.
train['QD'] = train['quality'] / (train['duration']+1)
test['QD'] = test['quality'] / (test['duration']+1)

### Encoding

In [8]:
train.fillna('(not set)', inplace=True)
test.fillna('(not set)', inplace=True)

In [9]:
# 학습데이터에 한하여 Target encoding 한다.
# keyword, referral_path
category = ['browser', 'device', 'traffic_source', 'traffic_medium','keyword2', 'referral_path2']
                        
for i in category:
    rate = train.groupby(i)['TARGET'].mean()
    train[i] = train[i].map(rate)
    test[i] = test[i].map(rate)
    test[i].fillna(train[i].mean(), inplace=True)

In [10]:
str_col = ['continent', 'new', 'OS', 'subcontinent', 'country', 'keyword1','referral_path1']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

### Feature Transformation

In [11]:
# sqrt transformation을 시도한다.
train['duration'] = train['duration']**0.5
test['duration'] = test['duration']**0.5

In [12]:
train['quality'] = train['quality']**0.5
test['quality'] = test['quality']**0.5

### Feature Selection

In [13]:
# ID열을 제거한다.
pred_test= test[['sessionID']]

# sessionID 변수를 제거한다.
train.drop(['sessionID'], axis=1, inplace=True)
test.drop(['sessionID'], axis=1, inplace=True)

In [24]:
unuse = ['userID']

X_train, y_train = train.drop(['TARGET']+unuse, axis=1), train['TARGET'].astype('i')
X_test = test.drop(unuse, axis=1)
tr_x, val_x, tr_y, val_y = train_test_split(X_train, y_train, test_size=0.3, random_state=2024)
model = LGBMRegressor(n_estimators=1000,  num_leaves=48,learning_rate=0.01,
                          eval_metric='RMSE',
                          random_state=2024)
model.fit(tr_x, tr_y)
print('RMSE(train):',mean_squared_error(model.predict(tr_x), tr_y)**0.5)
print('RMSE:', mean_squared_error(model.predict(val_x), val_y)**0.5)
print('(round)RMSE:', mean_squared_error(model.predict(val_x).round(), val_y)**0.5)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002284 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1448
[LightGBM] [Info] Number of data points in the train set: 89417, number of used features: 18
[LightGBM] [Info] Start training from score 6.105550
[LightGBM] [Warning] Unknown parameter: eval_metric
RMSE(train): 3.1227287248918043
[LightGBM] [Warning] Unknown parameter: eval_metric
RMSE: 3.697428138581109
[LightGBM] [Warning] Unknown parameter: eval_metric
(round)RMSE: 3.7090895985427834


In [ ]:
# Transform 생략

In [ ]:
RMSE(train): 3.1715002491716917
RMSE: 3.688607335828954
(round)RMSE: 3.7029391378962093'''

In [26]:
model.feature_importances_

array([1242, 2453, 1002, 1653, 8577, 9101,  446, 2129,  813, 2301, 2372,
       1393,  843, 1014,  331, 1415, 3135, 6780])

In [25]:
# Catboost를 base로 삼고 importance가 낮은 feature는 제거한다.
importance = pd.DataFrame({'feature': model.feature_names_,
                           'importance':model.feature_importances_})
importance

AttributeError: 'LGBMRegressor' object has no attribute 'feature_names_'

In [12]:
# Catboost를 base로 삼고 importance가 낮은 feature는 제거한다.
importance = pd.DataFrame({'feature': model.feature_names_,
                           'importance':model.feature_importances_})
importance

,feature,importance
0,browser,1.776307
1,OS,3.363068
2,device,1.084993
3,new,4.370651
4,quality,44.745599
5,duration,17.877549
6,transaction,0.880510
7,transaction_revenue,1.581597
8,continent,0.381199
9,subcontinent,10.366317


## <font color='forestgreen'> HyperParameter Tunning

In [27]:
X_train, y_train = train.drop(['TARGET']+unuse, axis=1), train['TARGET'].astype('i')
X_test = test.drop(unuse, axis=1)
model = LGBMRegressor(n_estimators=1000,  num_leaves=48,learning_rate=0.01,
                          eval_metric='RMSE',
                          random_state=2024)
model.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003323 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1466
[LightGBM] [Info] Number of data points in the train set: 127739, number of used features: 18
[LightGBM] [Info] Start training from score 6.116260


LGBMRegressor(eval_metric='RMSE', learning_rate=0.01, n_estimators=1000,
              num_leaves=48, random_state=2024)

In [28]:
pred_test['TARGET'] = model.predict(X_test)

[LightGBM] [Warning] Unknown parameter: eval_metric


## <font color='forestgreen'> Modeling
- HyperParameter Tunning은 수동으로 한다.

In [ ]:
lgbm

In [ ]:
cat

In [ ]:
knn

In [ ]:
dt

In [ ]:
rf

In [ ]:
br

In [ ]:
sgd

#### Ensemble

In [ ]:
model = VotingRegressor([('lr', r1), ('rf', r2), ('r3', r3)])
model.fit(X_train, y_train)
print('RMSE(train):',mean_squared_error(model.predict(tr_x), tr_y)**0.5)
print('RMSE:', mean_squared_error(model.predict(val_x), val_y)**0.5)
print('(round)RMSE:', mean_squared_error(model.predict(val_x).round(), val_y)**0.5)

In [ ]:
# 직접 비율을 달리하며 Ensemble한다.

In [ ]:
from sklearn import linear_model
>>> clf = linear_model.BayesianRidge()
>>> r1 = LinearRegression()
>>> r2 = RandomForestRegressor(n_estimators=10, random_state=1)
>>> r3 = KNeighborsRegressor()
>>> X = np.array([[1, 1], [2, 4], [3, 9], [4, 16], [5, 25], [6, 36]])
>>> y = np.array([2, 6, 12, 20, 30, 42])
>>> er = VotingRegressor([('lr', r1), ('rf', r2), ('r3', r3)])
>>> print(er.fit(X, y).predict(X))
[ 6.8...  8.4... 12.5... 17.8... 26...  34...]

## <font color='forestgreen'> Save data

In [29]:
sample = pd.read_csv('data/submission/sample_submission.csv')
sample = sample[['sessionID']].merge(pred_test, on='sessionID', how='left').fillna(1)
sample

,sessionID,TARGET
0,SESSION_252289,26.615789
1,SESSION_252290,1.000000
2,SESSION_252291,2.729949
3,SESSION_252292,4.265513
4,SESSION_252293,10.715498
...,...,...
79781,SESSION_332070,1.000000
79782,SESSION_332071,1.000000
79783,SESSION_332072,2.758734
79784,SESSION_332073,4.745194


In [30]:
date = str(pd.Timestamp.now())[:16].replace('-','').replace(' ','_').replace(':','')
filename = f'./data/submission/{date}'
sample.to_csv(f'{filename}.csv', index=False)
print(f'{filename} is saved.')

./data/submission/20240227_1641 is saved.
